# The webscraper to scrape 5 kinds of vehicles (car, bike, bus, plain, boat).

In [6]:
!pip install requests
!pip install BeautifulSoup4
!pip install selenium


[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/9.9 MB ? eta -:--:--
      --------------------------------------- 0.2/9.9 MB 5.0 MB/s eta 0:00:02
     -- ------------------------------------- 0.7/9.9 MB 7.0 MB/s eta 0:00:02
     ----- ---------------------------------- 1.5/9.9 MB 10.3 MB/s eta 0:00:01
     --------- ------------------------------ 2.5/9.9 MB 12.1 MB/s eta 0:00:01
     ------------- -------------------------- 3.3/9.9 MB 13.2 MB/s eta 0:00:01
     ----------------- ---------------------- 4.4/9.9 MB 13.9 MB/s eta 0:00:01
     -------------------- ------------------- 5.2/9.9 MB 14.4 MB/s eta 0:00:01
     ------------------------ --------------- 6.0/9.9 MB 14.9 MB/s eta 0:00:01
     --------------------------- ------------ 6.7/9.9 MB 15.4 MB/s eta 0:00:01
     --------------------------- ------------ 6.9/9.9 MB 13.8 MB/s eta 0:00:01
     ------------------------------- -------- 7.7/9.9 MB 14.5 MB/s eta 0:00:01
     ---------------------------------- ----- 8.4/9.9 MB 14.6 


[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
import os
import time
import requests
from selenium import webdriver

def download_images(url, folder_name):
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(5)
    elem = driver.find_element_by_link_text('Accept all')
    elem.click()
    time.sleep(5)
    elem = driver.find_element_by_link_text('Images')
    elem.click()
    value = 0
    for i in range(5):
        driver.execute_script("scrollBy("+ str(value) +",+1000);")
        value += 1000
        time.sleep(3)
    images = driver.find_elements_by_xpath('//img[@class="rg_i"]')
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    for i in range(len(images)):
        images[i].click()
        time.sleep(2)
        img_url = driver.find_element_by_xpath('//img[@class="n3VNCb"]')
        src = img_url.get_attribute('src')
        with open(f'{folder_name}/{i}.jpg', 'wb') as f:
            f.write(requests.get(src).content)
        time.sleep(2)
    driver.close()


vehicles = ['car', 'plane', 'bike', 'bus', 'boat']
for vehicle in vehicles:
    url = "https://www.google.com/imghp"+vehicle
    folder_name = "data/"+vehicle
    download_images(url, folder_name)

AttributeError: 'WebDriver' object has no attribute 'find_element_by_link_text'

In [19]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import urllib.request

# Set up Selenium web driver
driver = webdriver.Chrome()

# Navigate to Google Images
driver.get("https://www.google.com/imghp")

cookies = driver.find_element(By.ID,"L2AGLb")
cookies.click()
time.sleep(5)
# Search for images
search_box = driver.find_element(By.NAME,"q")
search_box.send_keys("cats")
search_box.submit()

# Extract image URLs
image_urls = []
for i in range(10):
    driver.execute_script("window.scrollBy(0,10000)")
    try:
        image_element = driver.find_element_by_xpath(f"//img[@class='rg_i'][{i+1}]")
        image_urls.append(image_element.get_attribute("src"))
    except:
        pass

# Download images
for i, url in enumerate(image_urls):
    urllib.request.urlretrieve(url, f"data/cat{i}.jpg")